# Outliers

In [1]:
import pandas as pd
import sys

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)

Python version 3.6.2 |Anaconda, Inc.| (default, Sep 19 2017, 08:03:39) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.20.3


In [3]:
# Create a dataframe with dates as your index
MLIs = ['A010', 'A010', 'A010', 'A010', '0010', '0010', '0026', '0026', '0010', '0010'] 
data = [1.0, 2, 3, 4, 5, 6, 7, 8, 9, 10]
idx = pd.date_range('1/1/2015', periods=10, freq='MS')
df1 = pd.DataFrame(data, index=idx, columns=['Duration'])
df1['MLI'] = MLIs

# Create a second dataframe
data2 = [10.0, 10.0, 9, 9, 8, 8, 7, 7, 6, 6]
idx2 = pd.date_range('1/1/2016', periods=10, freq='MS')
df2 = pd.DataFrame(data2, index=idx2, columns=['Duration'])
df2['MLI'] = MLIs

In [4]:
# Combine dataframes
df = pd.concat([df1,df2])
df

,Duration,MLI
2015-01-01,1.0,A010
2015-02-01,2.0,A010
2015-03-01,3.0,A010
2015-04-01,4.0,A010
2015-05-01,5.0,0010
2015-06-01,6.0,0010
2015-07-01,7.0,0026
2015-08-01,8.0,0026
2015-09-01,9.0,0010
2015-10-01,10.0,0010


# Ways to Calculate Outliers

Note: Average and Standard Deviation are only valid for gaussian distributions.

In [5]:
# Method 1

# make a copy of original df
newdf = df.copy()

newdf['x-Mean'] = abs(newdf['Duration'] - newdf['Duration'].mean())
newdf['1.96*std'] = 1.96*newdf['Duration'].std()  
newdf['Outlier'] = abs(newdf['Duration'] - newdf['Duration'].mean()) > 1.96*newdf['Duration'].std()
newdf

,Duration,MLI,x-Mean,1.96*std,Outlier
2015-01-01,1.0,A010,5.75,5.200273,True
2015-02-01,2.0,A010,4.75,5.200273,False
2015-03-01,3.0,A010,3.75,5.200273,False
2015-04-01,4.0,A010,2.75,5.200273,False
2015-05-01,5.0,0010,1.75,5.200273,False
2015-06-01,6.0,0010,0.75,5.200273,False
2015-07-01,7.0,0026,0.25,5.200273,False
2015-08-01,8.0,0026,1.25,5.200273,False
2015-09-01,9.0,0010,2.25,5.200273,False
2015-10-01,10.0,0010,3.25,5.200273,False


In [6]:
# Method 2
# Group by item

# make a copy of original df
newdf = df.copy()

State = newdf.groupby('MLI')

newdf['Outlier'] = State.transform( lambda x: abs(x-x.mean()) > 1.96*x.std() )
newdf['x-Mean'] = State.transform( lambda x: abs(x-x.mean()) )
newdf['1.96*std'] = State.transform( lambda x: 1.96*x.std() )
newdf

,Duration,MLI,Outlier,x-Mean,1.96*std
2015-01-01,1.0,A010,False,5.00,7.554813
2015-02-01,2.0,A010,False,4.00,7.554813
2015-03-01,3.0,A010,False,3.00,7.554813
2015-04-01,4.0,A010,False,2.00,7.554813
2015-05-01,5.0,0010,False,2.25,3.434996
2015-06-01,6.0,0010,False,1.25,3.434996
2015-07-01,7.0,0026,False,0.25,0.980000
2015-08-01,8.0,0026,False,0.75,0.980000
2015-09-01,9.0,0010,False,1.75,3.434996
2015-10-01,10.0,0010,False,2.75,3.434996


In [7]:
# Method 2
# Group by multiple items

# make a copy of original df
newdf = df.copy()

MLIMonth = newdf.groupby(['MLI', lambda x: x.month])

newdf['Outlier'] = MLIMonth.transform( lambda x: abs(x-x.mean()) > 1.96*x.std() )
newdf['x-Mean'] = MLIMonth.transform( lambda x: abs(x-x.mean()) )
newdf['1.96*std'] = MLIMonth.transform( lambda x: 1.96*x.std() )
newdf

,Duration,MLI,Outlier,x-Mean,1.96*std
2015-01-01,1.0,A010,False,4.5,12.473364
2015-02-01,2.0,A010,False,4.0,11.087434
2015-03-01,3.0,A010,False,3.0,8.315576
2015-04-01,4.0,A010,False,2.5,6.929646
2015-05-01,5.0,0010,False,1.5,4.157788
2015-06-01,6.0,0010,False,1.0,2.771859
2015-07-01,7.0,0026,False,0.0,0.000000
2015-08-01,8.0,0026,False,0.5,1.385929
2015-09-01,9.0,0010,False,1.5,4.157788
2015-10-01,10.0,0010,False,2.0,5.543717


In [8]:
# Method 3
# Group by item

# make a copy of original df
newdf = df.copy()

MLI = newdf.groupby('MLI')

def s(group):
    group['x-Mean'] = abs(group['Duration'] - group['Duration'].mean())
    group['1.96*std'] = 1.96*group['Duration'].std()  
    group['Outlier'] = abs(group['Duration'] - group['Duration'].mean()) > 1.96*group['Duration'].std()
    return group

Newdf2 = MLI.apply(s)
Newdf2

,Duration,MLI,x-Mean,1.96*std,Outlier
2015-01-01,1.0,A010,5.00,7.554813,False
2015-02-01,2.0,A010,4.00,7.554813,False
2015-03-01,3.0,A010,3.00,7.554813,False
2015-04-01,4.0,A010,2.00,7.554813,False
2015-05-01,5.0,0010,2.25,3.434996,False
2015-06-01,6.0,0010,1.25,3.434996,False
2015-07-01,7.0,0026,0.25,0.980000,False
2015-08-01,8.0,0026,0.75,0.980000,False
2015-09-01,9.0,0010,1.75,3.434996,False
2015-10-01,10.0,0010,2.75,3.434996,False


In [9]:
# Method 3
# Group by multiple items

# make a copy of original df
newdf = df.copy()

MLIMonth = newdf.groupby(['MLI', lambda x: x.month])

def s(group):
    group['x-Mean'] = abs(group['Duration'] - group['Duration'].mean())
    group['1.96*std'] = 1.96*group['Duration'].std()  
    group['Outlier'] = abs(group['Duration'] - group['Duration'].mean()) > 1.96*group['Duration'].std()
    return group

Newdf2 = MLIMonth.apply(s)
Newdf2

,Duration,MLI,x-Mean,1.96*std,Outlier
2015-01-01,1.0,A010,4.5,12.473364,False
2015-02-01,2.0,A010,4.0,11.087434,False
2015-03-01,3.0,A010,3.0,8.315576,False
2015-04-01,4.0,A010,2.5,6.929646,False
2015-05-01,5.0,0010,1.5,4.157788,False
2015-06-01,6.0,0010,1.0,2.771859,False
2015-07-01,7.0,0026,0.0,0.000000,False
2015-08-01,8.0,0026,0.5,1.385929,False
2015-09-01,9.0,0010,1.5,4.157788,False
2015-10-01,10.0,0010,2.0,5.543717,False


Assuming a non gaussian distribution (if you plot it, it will not look like a normal distribution)

In [10]:
# make a copy of original df
newdf = df.copy()

MLI = newdf.groupby('MLI')

newdf['Lower'] = MLI['Duration'].transform( lambda x: x.quantile(q=.25) - (1.5*(x.quantile(q=.75)-x.quantile(q=.25))) )
newdf['Upper'] = MLI['Duration'].transform( lambda x: x.quantile(q=.75) + (1.5*(x.quantile(q=.75)-x.quantile(q=.25))) )
newdf['Outlier'] = (newdf['Duration'] < newdf['Lower']) | (newdf['Duration'] > newdf['Upper']) 
newdf

,Duration,MLI,Lower,Upper,Outlier
2015-01-01,1.0,A010,-7.000,19.000,False
2015-02-01,2.0,A010,-7.000,19.000,False
2015-03-01,3.0,A010,-7.000,19.000,False
2015-04-01,4.0,A010,-7.000,19.000,False
2015-05-01,5.0,0010,2.625,11.625,False
2015-06-01,6.0,0010,2.625,11.625,False
2015-07-01,7.0,0026,6.625,7.625,False
2015-08-01,8.0,0026,6.625,7.625,True
2015-09-01,9.0,0010,2.625,11.625,False
2015-10-01,10.0,0010,2.625,11.625,False
